##UNet

In [ ]:
import torch
import torch.nn.functional as F


def conv3x3_bn(ci, co):
    return torch.nn.Sequential(
        # capa convolucional con filtro 3x3 y padding = 1, por lo que no cambian las dimensiones
        torch.nn.Conv2d(ci, co, 3, padding=1), # ci = canales de entrada; co 0 canales de salida
        torch.nn.BatchNorm2d(co),
        torch.nn.ReLU(inplace=True)
    )

def encoder_conv(ci, co):
  return torch.nn.Sequential(
        torch.nn.MaxPool2d(2),  # max pool 2x2
        conv3x3_bn(ci, co),     # conv 3x3, ReLU
        conv3x3_bn(co, co),     # conv 3x3, ReLU
    )

class deconv(torch.nn.Module):
    def __init__(self, ci, co):
        super(deconv, self).__init__()

        # upconv 2x2 (convolución traspuesta 2D)
        self.upsample = torch.nn.ConvTranspose2d(ci, co, 2, stride=2) # kernel de tamaño 2x2 y stride = 2 
        # con kernel 2x2 y stride = 2 se obtiene un mapa de características del doble de resolución

        self.conv1 = conv3x3_bn(ci, co) # conv 3x3, ReLU
        self.conv2 = conv3x3_bn(co, co) # conv 3x3, ReLU
    
    # recibe la salida de la capa anterior(x1) y la salida de la etapa
    # correspondiente del encoder (x2)
    def forward(self, x1, x2):
        x1 = self.upsample(x1)  # A la salida de la capa anterior se aplica convolución traspuesta

        # Para evitar inconcordancias en el tamaño y que las dimensiones sean las mismas
        diffX = x2.size()[2] - x1.size()[2]
        diffY = x2.size()[3] - x1.size()[3]
        x1 = F.pad(x1, (diffX, 0, diffY, 0))

        # concatenamos los tensores
        x = torch.cat([x2, x1], dim=1)
        x = self.conv1(x)
        x = self.conv2(x)
        return x


# n_classes número de clases diferentes
# in_ch número de canales de la imagen de entrada
class UNet(torch.nn.Module):
    def __init__(self, n_classes=3, in_ch=1): 
        super().__init__()

        # lista de capas en encoder-decoder con número de filtros
        c = [16, 32, 64, 128] # enconder en este orden; decoder en orden inverso

        # primera capa conv que recibe la imagen
        self.conv1 = torch.nn.Sequential(
          conv3x3_bn(in_ch, c[0]),
          conv3x3_bn(c[0], c[0]),
        )
        # capas del encoder
        self.conv2 = encoder_conv(c[0], c[1])
        self.conv3 = encoder_conv(c[1], c[2])
        self.conv4 = encoder_conv(c[2], c[3])

        # capas del decoder
        self.deconv1 = deconv(c[3],c[2])
        self.deconv2 = deconv(c[2],c[1])
        self.deconv3 = deconv(c[1],c[0])

        # útlima capa conv que nos da la máscara
        # el número de filtros a la entrada es el primer valor de la lista
        # se aplican tantos filtros como clases haya 
        self.out = torch.nn.Conv2d(c[0], n_classes, 3, padding=1)

    def forward(self, x):
        # encoder
        x1 = self.conv1(x)
        x2 = self.conv2(x1)
        x3 = self.conv3(x2)
        x = self.conv4(x3)
        # decoder
        x = self.deconv1(x, x3)
        x = self.deconv2(x, x2)
        x = self.deconv3(x, x1)
        x = self.out(x)
        return x

##UNet-ResNet

In [ ]:
class out_conv(torch.nn.Module):
    def __init__(self, ci, co, coo):
        super(out_conv, self).__init__()

        # upconv 2x2 (convolución traspuesta 2D)
        self.upsample = torch.nn.ConvTranspose2d(ci, co, 2, stride=2) # kernel de tamaño 2x2 y stride = 2
        # con kernel 2x2 y stride = 2 se obtiene un mapa de características del doble de resolución

        self.conv = conv3x3_bn(ci, co)  # conv 3x3, ReLU
        self.final = torch.nn.Conv2d(co, coo, 1) # A la salida se obtiene el número de clases necesario

    def forward(self, x1, x2):
        x1 = self.upsample(x1)
        diffX = x2.size()[2] - x1.size()[2]
        diffY = x2.size()[3] - x1.size()[3]
        x1 = F.pad(x1, (diffX, 0, diffY, 0))
        x = self.conv(x1)
        x = self.final(x)
        return x

class UNetResnet(torch.nn.Module):
    def __init__(self, n_classes=3, in_ch=1):
        super().__init__()

        # En lugar de utilizar un encoder propio, se utiliza una red entrenada: ResNet18
        self.encoder = torchvision.models.resnet18(pretrained=True)  

        # ResNet18 está diseñada para trabajar con 3 canaless, pero nuestras imagenes son en ByN 
        # por lo que es necesasrio modificar la primera capa    
        if in_ch != 3:
          self.encoder.conv1 = torch.nn.Conv2d(in_ch, 64, kernel_size=7, stride=2, padding=3, bias=False)


        self.deconv1 = deconv(512,256)
        self.deconv2 = deconv(256,128)
        self.deconv3 = deconv(128,64)

        # Capa que genera los mapas de salidas. Estos tienen que tener tantos canales 
        # como número de clases
        self.out = out_conv(64, 64, n_classes)


    def forward(self, x):
        x_in = torch.tensor(x.clone())
        x = self.encoder.relu(self.encoder.bn1(self.encoder.conv1(x)))
        x1 = self.encoder.layer1(x)
        x2 = self.encoder.layer2(x1)
        x3 = self.encoder.layer3(x2)
        x = self.encoder.layer4(x3)
        x = self.deconv1(x, x3)
        x = self.deconv2(x, x2)
        x = self.deconv3(x, x1)
        x = self.out(x, x_in)
        return x

# VDSR

In [ ]:
import torch
import torch.nn as nn
from math import sqrt

class Conv_ReLU_Block(nn.Module):
    def __init__(self):
        super(Conv_ReLU_Block, self).__init__()
        # capa convolucional con filtro 3x3, padding = 1 y stride = 1
        # bias = false: no se agrega sesgo aprendible a la salida 
        self.conv = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1, bias=False)
        self.relu = nn.ReLU(inplace=True)
        
    def forward(self, x):
        return self.relu(self.conv(x))
        
class VDSR(nn.Module):
    def __init__(self):
        super().__init__()

        # Capa residual formada por 18 capas de convolución
        self.residual_layer = self.make_layer(Conv_ReLU_Block, 18)

        # capa convolucional con filtro 3x3, padding = 1 y stride = 1
        # bias = false: no se agrega sesgo aprendible a la salida 
        self.input = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1, bias=False)

        # capa convolucional con filtro 3x3, padding = 1 y stride = 1
        # bias = false: no se agrega sesgo aprendible a la salida 
        self.output = nn.Conv2d(in_channels=64, out_channels=3, kernel_size=3, stride=1, padding=1, bias=False)
        self.relu = nn.ReLU(inplace=True)
    
        for m in self.modules():
            # ininstance() devuelve True si el objeto especificado es del tipo especificado
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, sqrt(2. / n))
                
    # Por parámetros se pasan un bloque de convolución(conv + relu) y el número de capas, para 
    # generar una secuencia de capas de convolución
    def make_layer(self, block, num_of_layer):
        layers = []
        for _ in range(num_of_layer):
            layers.append(block())
        return nn.Sequential(*layers)

    # La entrada pasa por una ReLU, luego por la capa residual (formada por un
    # determinado número de bloques de convolución), por la capa de salida y,
    # finalmente, a la salida se suma la entrada.   
    def forward(self, x):
        residual = x
        out = self.relu(self.input(x))
        out = self.residual_layer(out)
        out = self.output(out)
        out = torch.add(out,residual)
        return out
 

# SRGAN

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torch import Tensor

class ResidualConvBlock(nn.Module):
    def __init__(self, in_features):
        super(ResidualConvBlock, self).__init__()
        self.rc_block  = nn.Sequential(
            nn.Conv2d(in_features, in_features, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(in_features), # El segundo parámetro es ‎eps, un valor añadido al denominador para la estabilidad numérica‎
            nn.PReLU(),
            nn.Conv2d(in_features, in_features, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(in_features),
        )

    def forward(self, x):
        identity = x
        out = self.rc_block (x)
        out = out + identity
        return out


class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        # Primera capa convolucional
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=9, stride=1, padding=4), 
            nn.PReLU()
            )

        # Bloques residuales
        res_blocks = []
        for _ in range(16):
            res_blocks.append(ResidualConvBlock(64))
        self.res_blocks = nn.Sequential(*res_blocks)

        # Segunda capa convolucional
        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1, bias=False), 
            nn.BatchNorm2d(64)
            )

        # Bloque convolución upsampling (x4)
        self.upsampling = nn.Sequential(
            nn.Conv2d(64, 256, kernel_size=3, stride=1, padding=1),
            nn.PixelShuffle(2),
            nn.PReLU(),
            nn.Conv2d(64, 256, kernel_size=3, stride=1, padding=1),
            nn.PixelShuffle(2),
            nn.PReLU()
        )

        # Capa de salida
        self.conv3 = nn.Conv2d(64, 3, kernel_size=9, stride=1, padding=4)

        # Se inicializa los pesos 
        self.initialize_weights()  

    def forward(self, x):
        return self.forward_impl(x)

    def forward_impl(self, x):
        out1 = self.conv1(x)
        out = self.res_blocks(out1)
        out2 = self.conv2(out)
        out = out1 + out2
        out = self.upsampling(out)
        out = self.conv3(out)
        return out

    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
                m.weight.data *= 0.1
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                m.weight.data *= 0.1

class ContentLoss(nn.Module):
    def __init__(self):
        super(ContentLoss, self).__init__()

        # Se carga el modelo VGG19 preentrenador.
        vgg19 = models.vgg19(pretrained=True, num_classes=1000).eval()

        # Se extraen las primeras 36 capas del modelo VGG19 como content loss
        self.feature_extractor = nn.Sequential(*list(vgg19.features.children())[:36])

        # Se congelan los parámetros del modelo
        for parameters in self.feature_extractor.parameters():
            parameters.requires_grad = False

        # The preprocessing method of the input data. This is the VGG model preprocessing method of the ImageNet dataset.
        # self.register_buffer("mean", torch.Tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1))
        # self.register_buffer("std", torch.Tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1))

    def forward(self, sr, hr):
        # Standardized operations.
        # sr = (sr - self.mean) / self.std
        # hr = (hr - self.mean) / self.std

        # Find the feature map difference between the two images.
        loss = F.mse_loss(self.feature_extractor(sr), self.feature_extractor(hr))

        return loss    


class Discriminator(nn.Module):
    def __init__(self) -> None:
        super(Discriminator, self).__init__()
        self.features = nn.Sequential(
            # input size. (3) x 96 x 96
            nn.Conv2d(3, 64, (3, 3), (1, 1), (1, 1), bias=True),
            nn.LeakyReLU(0.2, True),
            # state size. (64) x 48 x 48
            nn.Conv2d(64, 64, (3, 3), (2, 2), (1, 1), bias=False),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, True),
            nn.Conv2d(64, 128, (3, 3), (1, 1), (1, 1), bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, True),
            # state size. (128) x 24 x 24
            nn.Conv2d(128, 128, (3, 3), (2, 2), (1, 1), bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, True),
            nn.Conv2d(128, 256, (3, 3), (1, 1), (1, 1), bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, True),
            # state size. (256) x 12 x 12
            nn.Conv2d(256, 256, (3, 3), (2, 2), (1, 1), bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, True),
            nn.Conv2d(256, 512, (3, 3), (1, 1), (1, 1), bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, True),
            # state size. (512) x 6 x 6
            nn.Conv2d(512, 512, (3, 3), (2, 2), (1, 1), bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, True)
        )

        self.classifier = nn.Sequential(
            nn.Linear(512*16*16, 1024),
            nn.LeakyReLU(0.2, True),
            nn.Linear(1024, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        out = self.features(x)
        out = torch.flatten(out, 1)
        out = self.classifier(out)

        return out